## Sentinel-1 large-scale time-series/timescan processing

This note book is part 2 of the time-series/timescan creation. It requires that you already did the data inventory and download.

Also note that the minimum requirements are 16GB of RAM as well as enough space for the end products.

**The processing is split into 4 parts:**
- **Step 1:** Creation of single Analysis-Ready-Data products
- **Step 2:** Creation of time-series for each track
- **Step 3:** Creation of time-series mosaics (optional, i.e. only in case of more than 1 track within the AOI)
- **Step 4:** Creation of timescans (optional)

### 1) import python libs

In [ ]:
# standard libs
import os, glob, shutil, datetime

# ost libs
from ost.s1 import refine, grd2Ard, ts, grdBatch
from ost.helpers import raster as ras
from ost.helpers import vector as vec

### 2) re-define project structure 

Note that this should be in line with the directories defined in  part 1 of the time-series creation. In addition we need to define a temporary folder.

In [ ]:
# define a directory for storing temporary files during processing
tmpDir = '/path/to/tmp'

# the main project directory
prjDir = '/Phi_avollrath_AddSpace/thuerGrd'

# this is where we download the original scenes
dwnDir = '{}/download'.format(prjDir)

# this folder will be used for the inventory shape files
invDir = '{}/inventory'.format(prjDir)

# this folder will be used for the processed data
prcDir = '{}/processed'.format(prjDir)



### 3) define processing parameters

In [ ]:
# shapefile or ISO 3 code AOI
aoi = '/path/to/aoi'

# processing parameters
outResolution = 50                          # resolution of the output product in meters
lsMask = True                              # layover/shadow mask generation (Boolean)
spkFlt = False                               # speckle filtering (Boolean)

# 3 different output product type are available 
#  - RTC: radiometrically corrected for slope
#  - GTCgamma: geometrically terrain corrected and compensated for ellipsoid curvature
#  - GTCsigma: geometrically terrain corrected
outPrdType = 'GTCgamma'

# timeseries options
Timeseries = True
mtSpkFlt = True
toDB = True
dType = 'float32'

# timescan option
Timescan = True
metrics = ["avg", "max", "min", "std", "cov" ]
toPower = True
rescale = True
outRem = True

procParam = grdBatch.createProcParamDict(outResolution, lsMask, spkFlt, outPrdType,
                                         mtSpkFlt, metrics, toDB, dType, 
                                         toPower, rescale, outRem)

### 4) load refined data inventory file into a dataframe and visualize

In [ ]:
%matplotlib inline

# choose the orbit and pol info with most msoaics based (this comes from the data inventory)
mosaicKey = 'ASCENDING_VVVH'
invFile = glob.glob('{}/*_{}.shp'.format(invDir, mosaicKey))[0]
fpDataFrame = refine.readS1Inventory(invFile) # function to convert the input to GeoDataFrame
vec.plotInv(aoi, fpDataFrame)

### 5) Single scene processing

In the following cell we will trigger the single scene batch processing. The command will process all downloaded data and store it in the process folder under the "track number"/"acquisiton date". Frames of the same acqusition are merged to a single scene. During processing, intermediate products will be stored in a self-defined tmp folder (see above). This folder should accomodate enough space for the products, that in the case of multiple frames per acquisition can use a considerable amount of disk sapce.
    
Dependent on the amount of products to process, this can take while. 

In [ ]:
grdBatch.grd2ArdBatch(fpDataFrame, dwnDir, prcDir, tmpDir, procParam)
print('we can delete the downlaoded data')

### 6) Time-series processing

In this step the single scenes of each track are stacked, and multi-temporal filtered (if selected).
The output will no longer be the dimap SNAP format, but single geotiffs that are accompanied by a vrt file (e.g. Timeseries.VV.vrt) that allows to load the full stack for the respective polarisation. The band numbers inside the vrt stack refer to the numbers, with which the single files are labelled.

In [ ]:
grdBatch.ard2Ts(fpDataFrame, prcDir, tmpDir, procParam)
print('we can delete the singel TC products')

### 7) Create a time-series animation

While it is nice to have an animated image, this also helps to see if the processing went fine up to this stage. 


In [ ]:
procDict = refine.createProcDict(fpDataFrame)
for track, allScenes in procDict.items():

    trackDir = '{}/{}/Timeseries'.format(prcDir, track)
    outFile = '{}/ts_animation.gif'.format(trackDir)
    ts.createTsAnimation(trackDir, tmpDir, outFile, 100)

### 8) Create Timescan

In this step, each of the time-series (e.g. by track and polarisation) are used to create the multi-temporal statistics referred to as timescans.

In [ ]:
if Timescan is True:
    grdBatch.ts2Timescan(fpDataFrame, prcDir, procParam)
if Timeseries is False:
    print('we can delete all timeseries folders')
print('only mosaicking is missing')

### 9) Mosaicking

In this step we mosaic the timescans as well as the time-series (if selected) to seampless mocsiacs for the full Area of interest.

In [ ]:
grdBatch.ts2Mosaic(fpDataFrame, prcDir, tmpDir, Timeseries, Timescan)